In [ ]:
import pandas as pd
import numpy as np
import rdflib
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

## Loading knowledge graph

In [ ]:
graph = rdflib.Graph()
graph.parse("data/reference-kg.nt")

training_data = rdflib.Graph()
training_data.parse("data/fokg-sw-train-2024.nt")

## Converting to pykeen format

In [ ]:
data = np.array([(s, p, o) for s, p, o in graph])
pykeen_data = TriplesFactory.from_labeled_triples(data)


## Learning the Embeddings

In [ ]:
training, testing, validation = pykeen_data.split([0.8, 0.1, 0.1])
result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='TransE',
)
